In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('weather_data_month.csv', sep=',').drop(['text'], axis='columns')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11874 entries, 0 to 11873
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   января    11874 non-null  int64
 1   февраля   11874 non-null  int64
 2   марта     11874 non-null  int64
 3   апреля    11874 non-null  int64
 4   мая       11874 non-null  int64
 5   июня      11874 non-null  int64
 6   июля      11874 non-null  int64
 7   августа   11874 non-null  int64
 8   сентября  11874 non-null  int64
 9   октября   11874 non-null  int64
 10  ноября    11874 non-null  int64
 11  декабря   11874 non-null  int64
 12  month     11874 non-null  int64
dtypes: int64(13)
memory usage: 1.2 MB


In [5]:
X = data.drop('month', axis=1)
y = data['month'].astype('int64')

In [6]:
X.shape, y.shape

((11874, 12), (11874,))

In [7]:
from sklearn.tree import DecisionTreeClassifier

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.3, 
                                                      random_state=42)

In [9]:
dtc = DecisionTreeClassifier(random_state=17, max_depth=12 )

In [20]:
from sklearn.model_selection import  cross_val_score
np.mean(cross_val_score(dtc, X_train, y_train, cv=5))

0.9800266723976865

In [21]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [22]:
results = cross_val_score(dtc, X, y, cv=skf)

In [23]:
print("CV accuracy score: {:.2f}%".format(results.mean()*100))

CV accuracy score: 98.03%
